# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones
## Programación Distribuida sobre Grandes Volúmenes de Datos

Damián Barsotti 

### Facultad de Matemática Astronomía Física y Computación
## Universidad Nacional de Córdoba

<img src="http://program.ar/wp-content/uploads/2018/07/logo-UNC-FAMAF.png" alt="Drawing" style="width:80%;"/>

Antes de comenzar
En máquina virtual usando zeppeling

1. Lanzar terminal
1. Actualizar repo:
```sh
cd diplodatos_bigdata
git pull
```
1. Lanzar [Zeppelin](http://zeppelin.apache.org/):
```sh
cd
cd spark/zeppelin-0.8.3-bin-all
./bin/zeppelin-daemon.sh start
```
1. En navegador abrir [http://localhost:8080](http://localhost:8080)
1. Seleccionar `Import note`
1. Elegir json en `diplodatos_bigdata/clases/03_sql/note.json`
2. Seleccionar `Clase 03 - SQL`

# Datasets/Dataframes

* Spark permite interactuar con datos estructurados (Bases de Datos tabulares) o semiestructurados (JSON) con su componente **Spark SQL**.
* Sus interfaces son **SQL** y **Dataframe/Dataset** API .
    - Programática, parecida a [Python Pandas dataframes](http://pandas.pydata.org/pandas-docs/stable/dsintro.html).
    - Demasiado parecida. Ver [Koalas](https://github.com/databricks/koalas).
* La API Dataset es tipada y solo existe para Scala y Java.


<img src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/03_sql/unified_stack.png" alt="Drawing" style="width:70%;"/>

### API 2.x.x unificada

<img src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/03_sql/dataset_dataframe_unificado.png" alt="Drawing" style="width:70%;"/>


### SparkSession

* Para acceder al cluster desde la API se utiliza `SparkSession`.
* El `SparkContext` deriva de él.

```python
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark ejemplo") \
    .config("spark.some.config.option", "algun-valor") \
    .getOrCreate()
    
sc = spark.sparkContext

```

* En Zeppelin ya están predefinidos: 
  - `SparkSession` objeto `spark` 
  - `SparkContext` objeto `sc`


In [1]:
from os.path import join, abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("03_sql") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.sql.catalogImplementation", "hive")\
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

## Lectura de datos

### Estructurados y semiestructurados

#### Formatos:

* json
* csv
* parquet
* orc
* libsvm
* text
* ...

#### Fuentes de datos:

* Archivos en fs local o distribuid (ej hdfs)
* jdbc (posgress, oracle, mysql,...)
* Apache Hive (se usa execution backend Spark en ves de MR)
* Amazon Redshift, S3
* Azure Storage Services
* Cassandra
* MongoDB
* Neo4j
* ...


### Ejemplo

#### Tabla de perfiles [last.fm](last.fm)

Formato:

```
id \t gender ('m'|'f'|empty) \t age (int|empty) \t country (str|empty) \t registered (date|empty)
```

In [3]:
!head ../inputs/ds/userid-profile.tsv

id	gender	age	country	registered
user_000001	m		Japan	Aug 13, 2006
user_000002	f		Peru	Feb 24, 2006
user_000003	m	22	United States	Oct 30, 2005
user_000004	f			Apr 26, 2006
user_000005	m		Bulgaria	Jun 29, 2006
user_000006		24	Russian Federation	May 18, 2006
user_000007	f		United States	Jan 22, 2006
user_000008	m	23	Slovakia	Sep 28, 2006
user_000009	f	19	United States	Jan 13, 2007


Ver en `inferSchema` siguiente si `inferSchema=False`
> todo los datos son tomados como `string`

In [4]:
profiles = spark.read.load(
    "../inputs/ds/userid-profile.tsv", format="csv",
    delimiter="\t", header=True, inferSchema=True)

In [5]:
profiles.printSchema()

root
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- registered: string (nullable = true)



In [6]:
profiles.show()

+-----------+------+----+------------------+------------+
|         id|gender| age|           country|  registered|
+-----------+------+----+------------------+------------+
|user_000001|     m|null|             Japan|Aug 13, 2006|
|user_000002|     f|null|              Peru|Feb 24, 2006|
|user_000003|     m|  22|     United States|Oct 30, 2005|
|user_000004|     f|null|              null|Apr 26, 2006|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|
|user_000006|  null|  24|Russian Federation|May 18, 2006|
|user_000007|     f|null|     United States|Jan 22, 2006|
|user_000008|     m|  23|          Slovakia|Sep 28, 2006|
|user_000009|     f|  19|     United States|Jan 13, 2007|
|user_000010|     m|  19|            Poland| May 4, 2006|
|user_000011|     m|  21|           Finland| Sep 8, 2005|
|user_000012|     f|  28|     United States|Mar 30, 2005|
|user_000013|     f|  25|           Romania|Sep 25, 2006|
|user_000014|  null|null|              null|Jan 27, 2006|
|user_000015| 

### Query SQL plano

In [7]:
profiles.createOrReplaceTempView("users")

#Cantidad de usuarios por país
nUsr4Ctry = spark.sql(
    "SELECT country, count(*) AS cantidad FROM users GROUP BY country ORDER BY cantidad DESC")

nUsr4Ctry.show()

+------------------+--------+
|           country|cantidad|
+------------------+--------+
|     United States|     228|
|    United Kingdom|     126|
|              null|      85|
|            Poland|      50|
|           Germany|      36|
|            Norway|      35|
|           Finland|      32|
|            Canada|      32|
|            Turkey|      28|
|             Italy|      27|
|            Sweden|      24|
|       Netherlands|      23|
|         Australia|      22|
|Russian Federation|      22|
|            Brazil|      20|
|             Spain|      17|
|            France|      14|
|            Mexico|      12|
|           Belgium|       9|
|         Argentina|       9|
+------------------+--------+
only showing top 20 rows



### Query SQL programático

In [8]:
from pyspark.sql.functions import count

nUsr4Ctry2 = profiles \
                .groupBy("country").agg(count("*").alias("cantidad")) \
                .orderBy("cantidad", ascending=False)
# Cada operación SQL es un método

nUsr4Ctry2.show()

+------------------+--------+
|           country|cantidad|
+------------------+--------+
|     United States|     228|
|    United Kingdom|     126|
|              null|      85|
|            Poland|      50|
|           Germany|      36|
|            Norway|      35|
|           Finland|      32|
|            Canada|      32|
|            Turkey|      28|
|             Italy|      27|
|            Sweden|      24|
|       Netherlands|      23|
|         Australia|      22|
|Russian Federation|      22|
|            Brazil|      20|
|             Spain|      17|
|            France|      14|
|            Mexico|      12|
|         Argentina|       9|
|           Belgium|       9|
+------------------+--------+
only showing top 20 rows



#### Ejercicio

Complete los siguientes programas que calculan en un Dataframe 
la cantidad de usuarios por pais desagregando por sexo y 
ordenando por la cantidad de mayor a menor, usando **SQL plano y programático**.

In [9]:
# Con SQL plano

nUsr4CtryGen = spark.sql(
    "SELECT country, gender, count(*) AS cantidad FROM users GROUP BY country, gender ORDER BY cantidad DESC")

nUsr4CtryGen.show()

+--------------+------+--------+
|       country|gender|cantidad|
+--------------+------+--------+
| United States|     m|     113|
| United States|     f|     104|
|United Kingdom|     m|      81|
|          null|  null|      49|
|United Kingdom|     f|      34|
|        Poland|     f|      29|
|          null|     f|      23|
|       Germany|     m|      22|
|        Poland|     m|      19|
|        Turkey|     m|      18|
|        Canada|     m|      18|
|       Finland|     m|      17|
|        Norway|     f|      15|
|        Sweden|     m|      15|
|        Canada|     f|      14|
|         Italy|     m|      14|
|       Germany|     f|      14|
|       Finland|     f|      13|
|         Italy|     f|      13|
|   Netherlands|     m|      13|
+--------------+------+--------+
only showing top 20 rows



In [10]:
# Con SQL porgramático

nUsr4CtryGen2 = profiles \
                .groupBy("country", "gender").agg(count("*").alias("cantidad")) \
                .orderBy("cantidad", ascending=False)

nUsr4CtryGen2.show()

+--------------+------+--------+
|       country|gender|cantidad|
+--------------+------+--------+
| United States|     m|     113|
| United States|     f|     104|
|United Kingdom|     m|      81|
|          null|  null|      49|
|United Kingdom|     f|      34|
|        Poland|     f|      29|
|          null|     f|      23|
|       Germany|     m|      22|
|        Poland|     m|      19|
|        Turkey|     m|      18|
|        Canada|     m|      18|
|       Finland|     m|      17|
|        Sweden|     m|      15|
|        Norway|     f|      15|
|        Canada|     f|      14|
|       Germany|     f|      14|
|         Italy|     m|      14|
|          null|     m|      13|
|       Finland|     f|      13|
|   Netherlands|     m|      13|
+--------------+------+--------+
only showing top 20 rows



#### Lectura desde JDBC (Java Database Connectivity)

```python
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/test") \
    .option("dbtable", "projects") \
    .option("user", "username") \
    .option("password", "password") \
    .load()
```
Más información en:

* [Spark SQL](http://spark.apache.org/docs/latest/sql-programming-guide.html#jdbc-to-other-databases).
* [Ejemplo](https://supergloo.com/spark-sql/spark-sql-mysql-python-example-jdbc/).

#### Lectura desde Apache Hive 

```scala
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Ejemplo Spark Hive") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

sqlDF = spark.sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")
```
Más información en:

* [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html#hive-tables).

#### SQL

In [ ]:
spark.sql("drop table if exists mytable") # borro la tabla si existe

spark.sql("create table mytable as select * from users")
# Simula Hive Data Warehouse local

#### Dataframe

In [11]:
profiles.write.mode("overwrite").save("./profiles.parquet")

### Ejercicio

Complete el siguiente programa par calcular la edad promedio por género 
y guarde el resultado como tabla SQL y como archivo parquet.

#### SQL

In [ ]:
spark.sql("drop table if exists gen_prom") # borro la tabla si existe

spark.sql("create table gen_prom as SELECT gender, avg(age) AS age_avg FROM users GROUP BY gender")

#Cargo tabla y muestro su contenido

spark.sql("select * from gen_prom").show()

#### DataFrame

In [12]:
from pyspark.sql.functions import *

genProm = profiles \
            .groupBy('gender').agg(avg('age').alias("age_avg"))

genProm.write.mode("overwrite").save("./gen_prom.parquet")

# Cargo parquet y muestro su contenido
spark.read.load("./gen_prom.parquet").show()

+------+------------------+
|gender|           age_avg|
+------+------------------+
|     m|25.630573248407643|
|     f| 24.13157894736842|
|  null|              32.0|
+------+------------------+



Mas info

* [API Python SQL](http://spark.apache.org/docs/2.2.1/api/python/pyspark.sql.html)
* [Function Reference](http://spark.apache.org/docs/2.2.1/api/python/pyspark.sql.html#module-pyspark.sql.functions)
* [Doc Spark SQL](http://spark.apache.org/docs/2.2.1/sql-programming-guide.html)

## Eficiencia
---

Los **RDD** tienen el overhead de la *serialización*:

* cuando los objetos se transfieren (por red) y guardan (disco)
* overhead de garbage collector

Los **Datasets** solucionan estos problemas:

* Serializa a binario usando **encoders**
    - parte del proyecto Tungsten
    - permite operaciones sin deserializar
    - corre *off-heap* (sin garbage collection)
    - código para serialización generado en forma dinámica
* Con la información de la estructura (*schema*) Spark hace optimizaciones.
    - Usa *Catalyst optimizer*.
    - Transfiere solo columnas usadas, no objetos enteros (relational query plan).

#### WordCount con RDD

In [13]:
linesRDD = sc.textFile("../inputs/README.md")

wordsRDD = linesRDD \
            .flatMap(lambda l: l.split(" ")) \
            .filter(lambda w:  w)
#MapReduce:
wordCountRDD = wordsRDD.map(lambda w: (w,1)) \
                .reduceByKey(lambda nx,ny:  nx+ny)

resultRDD = wordCountRDD \
                .sortBy((lambda p: p[1]), ascending = False)
                # ordena por cantidad

print("Resultado:")

for w, c in resultRDD.collect()[:5]: #  traigo resultados
    print(w, c)

Resultado:
from 4
Apache 3
Zeppelin 3
and 3
to 3


<img src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/03_sql/Distributed-Wordcount-Chart.png" alt="Drawing" style="width:70%;"/>

#### WordCount con DataFrames

In [14]:
from pyspark.sql.functions import split, explode

linesDF = spark.read.text("../inputs/README.md").toDF("lineas")

wordsDF = linesDF \
            .select(explode(split("lineas", ' ')).alias("words")) \
            .filter("words != ''")

wordCountDF = wordsDF \
                .groupBy("words").count()

resultDF = wordCountDF \
                .orderBy("count", ascending=False)
#                // ordena por cantidad

print("Resultado:")

resultDF.show(n=5, truncate=False)

Resultado:
+--------+-----+
|words   |count|
+--------+-----+
|from    |4    |
|and     |3    |
|to      |3    |
|Apache  |3    |
|Zeppelin|3    |
+--------+-----+
only showing top 5 rows



<img src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/03_sql/Memory-Usage-when-Caching-Chart.png" alt="Drawing" style="width:70%;"/>

### Tungsten en acción

In [15]:
import math 

ints = range(int(math.pow(10, 6)))
print(ints[:10])
intsRDD = sc.parallelize(ints).setName("intsRDD").cache()

# Fuerzo evaluacion
print (intsRDD.count())

range(0, 10)
1000000


In [16]:
# Ver sparkui storage. Descomentar proximas lineas y ver de nuevo
from pyspark.sql.types import IntegerType

intsDF = spark.createDataFrame(ints, IntegerType()).cache()
intsDF.cache()

print (intsDF.count())

1000000


Para la segunda tarea, se demora mas porque se ejecutan en mas etapas

#### Ejemplo

In [17]:
df = spark.read.json("../inputs/ds/people.json")

# Displays the content of the DataFrame to stdout
df.show()

# Selecciona todo incrementando la edad
df.selectExpr("name", "age + 1").show()

# O tambien
df.select("name", df.age + 1).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [18]:
# Selecciona personas con mas de 21 años
df.filter("age > 21").show()
# Cuenta personas por edad
df.groupBy("age").count().show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



## Ventajas/desventajas de las diferentes APIs
---

### APIs tipadas y no tipadas


| Lenguaje  | Abstracción Principal  |
|---|---|
|Scala   | Dataset[T] y Dataframe (Datset[Row])  |
|Python   | Dataset[T]  |
| R  | Dataframe  |

#### Detección de errores

<img 
src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/03_sql/type-safety-spectrum.png" alt="Drawing" style="width:70%;"/>

### Cuando usar Datasets Dataframes o RDD

* Si se necesita expresiones de alto nivel, filters, maps, aggregations, promedios, sumatorias, queries SQL, acceso por columna y funciones lambda sobre datos semiestructurados
    - para desarrollar aplicaciones finales (Data Ingeeniering) usar **Datasets**.
    - para análisis interactivo (Data Scientist) usar **Dataframes**. 
* Si se necesita mayor seguridad de tipos chequeandolos a tiempo de compilación, objetos JVM, beneficios de optimización Catalyst y código eficiente con Tungsten usar **Datasets**.
* Si se quiere una API unificada a traves de la la librerías Spark usar **DataFrames** o **Datasets**.
* Si se quiere trabajar en R no queda otra que usar **DataFrames**.
* Si se quiere trabajar en Python no queda otra que usar **DataFrames** y recurrir a **RDDs** si se necesita mayor control.


In [19]:
!head ../inputs/ds/userid-profile.tsv

id	gender	age	country	registered
user_000001	m		Japan	Aug 13, 2006
user_000002	f		Peru	Feb 24, 2006
user_000003	m	22	United States	Oct 30, 2005
user_000004	f			Apr 26, 2006
user_000005	m		Bulgaria	Jun 29, 2006
user_000006		24	Russian Federation	May 18, 2006
user_000007	f		United States	Jan 22, 2006
user_000008	m	23	Slovakia	Sep 28, 2006
user_000009	f	19	United States	Jan 13, 2007


In [23]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

profiles = spark.read.load("../inputs/ds/userid-profile.tsv",
                    format="csv", delimiter="\t", header=True, inferSchema=True)
profiles.show()

+-----------+------+----+------------------+------------+
|         id|gender| age|           country|  registered|
+-----------+------+----+------------------+------------+
|user_000001|     m|null|             Japan|Aug 13, 2006|
|user_000002|     f|null|              Peru|Feb 24, 2006|
|user_000003|     m|  22|     United States|Oct 30, 2005|
|user_000004|     f|null|              null|Apr 26, 2006|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|
|user_000006|  null|  24|Russian Federation|May 18, 2006|
|user_000007|     f|null|     United States|Jan 22, 2006|
|user_000008|     m|  23|          Slovakia|Sep 28, 2006|
|user_000009|     f|  19|     United States|Jan 13, 2007|
|user_000010|     m|  19|            Poland| May 4, 2006|
|user_000011|     m|  21|           Finland| Sep 8, 2005|
|user_000012|     f|  28|     United States|Mar 30, 2005|
|user_000013|     f|  25|           Romania|Sep 25, 2006|
|user_000014|  null|null|              null|Jan 27, 2006|
|user_000015| 

In [35]:
regByDayOfWeek = profiles.select(
    "registered", unix_timestamp("registered", format="MMM d, yyyy").alias("reg_sec")) \
        .select("*", from_unixtime('reg_sec',"E").alias("day_week"))

regByDayOfWeek.groupBy("day_week").count().show()

+--------+-----+
|day_week|count|
+--------+-----+
|     Sun|  148|
|    null|    8|
|     Mon|  147|
|     Thu|  131|
|     Sat|  125|
|     Wed|  155|
|     Fri|  131|
|     Tue|  147|
+--------+-----+



Fin